In [2]:
import visual_behavior.validation.sdk as sdk_validation
from visual_behavior.validation.sdk import ValidateSDK
from visual_behavior import database as db
# import visual_behavior.visualization.qc.data_loading as dl
import visual_behavior.data_access.loading as dl
import visual_behavior.utilities as vbu
import h5py
%widescreen

In [3]:
conn = db.Database('visual_behavior_data')
res = conn['sdk_validation']['validation_results'].find({'eye_tracking':0})
eye_tracking_failures = pd.DataFrame(list(res))
conn.close()

In [4]:
len(eye_tracking_failures)

248

In [5]:
def get_workflow_state(bsid):
    query = '''
    select oe.workflow_state
    from behavior_sessions bs
    join ophys_experiments oe on oe.ophys_session_id = bs.ophys_session_id
    where bs.id = {}
    '''.format(bsid)
    ans = db.lims_query(query)
    if isinstance(ans, pd.DataFrame):
        return 'passed' if (ans['workflow_state'] == 'passed').any() else 'failed'
    else:
        return ans
    
def get_path(well_known_files, name):
    '''
    get path from well_known_files dataframe
    inputs:
        well_known_files (dataframe with index set to name)
        name (string with desired name)
    returns:
        string with full filepath
    '''
    if name in well_known_files.index:
        return ''.join(well_known_files.loc[name][['storage_directory', 'filename']].tolist())
    else:
        return None
    
def get_movie_frame_count(ophys_session_id,movie_type='EyeTracking'):
    wkf = db.get_well_known_files(ophys_session_id)
    movie = vbu.Movie(get_path(wkf,'Raw{}Video'.format(movie_type)))
    return movie.frame_count

def get_videomon_frame_count(ophys_session_id,movie_type='EyeTracking'):
    wkf = db.get_well_known_files(ophys_session_id)
    movie_path = get_path(wkf,'Raw{}Video'.format(movie_type))
    if os.path.exists(movie_path.replace('.avi','.h5')):
        f = h5py.File(movie_path.replace('.avi','.h5'), 'r')
        frame_intervals = f['frame_intervals'][:]
        return len(frame_intervals) + 1
    else:
        return 0

def get_sync_edge_count(ophys_session_id,movie_type='EyeTracking'):
    wkf = db.get_well_known_files(ophys_session_id)
    sync = vbu.get_sync_data(get_path(wkf,'OphysRigSync'))
    
    if movie_type == 'EyeTracking':
        line = 'cam2_exposure_rising' if 'cam2_exposure_rising' in sync.keys() else 'eye_tracking_rising'
    elif movie_type == 'BehaviorTracking':
        line = 'cam1_exposure_rising' if 'cam1_exposure_rising' in sync.keys() else 'behavior_monitoring_rising'
        
    
    sync_frame_times = sync[line]
    return len(sync_frame_times)

In [6]:
filtered_ophys_session_table = dl.get_filtered_ophys_sessions_table()

AttributeError: module 'visual_behavior.data_access.loading' has no attribute 'get_filtered_ophys_sessions_table'

In [158]:
filtered_sessions_no_eye_tracking = filtered_ophys_session_table.merge(
    eye_tracking_failures,
    left_on = 'behavior_session_id',
    right_on = 'behavior_session_id',
    how='inner'
).drop_duplicates('ophys_session_id')
len(filtered_sessions_no_eye_tracking)

52

In [159]:
filtered_sessions_no_eye_tracking['workflow_state'] = filtered_sessions_no_eye_tracking['behavior_session_id'].map(lambda bsid: get_workflow_state(bsid))
for video in ['EyeTracking','BehaviorTracking']:
    filtered_sessions_no_eye_tracking['{}_movie_frame_count'.format(video)] = filtered_sessions_no_eye_tracking['ophys_session_id'].map(lambda osid: get_movie_frame_count(osid,video))
    filtered_sessions_no_eye_tracking['{}_videomon_frame_count'.format(video)] = filtered_sessions_no_eye_tracking['ophys_session_id'].map(lambda osid: get_videomon_frame_count(osid,video))
    filtered_sessions_no_eye_tracking['{}_sync_pulse_count'.format(video)] = filtered_sessions_no_eye_tracking['ophys_session_id'].map(lambda osid: get_sync_edge_count(osid,video))
    

In [160]:
def likely_line_switch(row):
    if (
        row['EyeTracking_movie_frame_count'] == 
        row['EyeTracking_videomon_frame_count'] == 
        row['BehaviorTracking_sync_pulse_count'] and
        row['BehaviorTracking_movie_frame_count'] == 
        row['BehaviorTracking_videomon_frame_count'] == 
        row['EyeTracking_sync_pulse_count']
        ):
        return True
    else:
        return False

In [161]:
def missing_eye_video_frames(row):
    if (
        row['EyeTracking_sync_pulse_count'] ==
        row['EyeTracking_videomon_frame_count'] and
        row['EyeTracking_movie_frame_count'] <
        row['EyeTracking_sync_pulse_count'] and
        row['likely_line_switch'] == False
        ):
        return True
    else:
        return False
    
def appears_fine_but_still_failing(row):
    if row['EyeTracking_sync_pulse_count'] == row['EyeTracking_movie_frame_count']:
        return True
    else:
        return False
    
def missing_vidoemon_log(row):
    if row['EyeTracking_videomon_frame_count'] == 0:
        return True
    else:
        return False
    
def mystery_failure(row):
    if (
        row['likely_line_switch'] == False and
        row['missing_eye_video_frames'] == False and
        row['appears_fine_but_still_failing'] == False and
        row['missing_vidoemon_log'] == False
        ):
        return True
    else:
        return False
    
filtered_sessions_no_eye_tracking['likely_line_switch'] = filtered_sessions_no_eye_tracking.apply(likely_line_switch, axis=1)
filtered_sessions_no_eye_tracking['missing_eye_video_frames'] = filtered_sessions_no_eye_tracking.apply(missing_eye_video_frames, axis=1)
filtered_sessions_no_eye_tracking['appears_fine_but_still_failing'] = filtered_sessions_no_eye_tracking.apply(appears_fine_but_still_failing, axis=1)
filtered_sessions_no_eye_tracking['missing_vidoemon_log'] = filtered_sessions_no_eye_tracking.apply(missing_vidoemon_log, axis=1)
filtered_sessions_no_eye_tracking['mystery_failure'] = filtered_sessions_no_eye_tracking.apply(mystery_failure, axis=1)

In [162]:
cols = [
    'behavior_session_id',
    'ophys_session_id',
    'ophys_experiment_id',
    'project_code',
    'workflow_state',
    'equipment_name',
    'date_of_acquisition'
]
cols = cols + [
    'EyeTracking_movie_frame_count',
    'EyeTracking_videomon_frame_count',
    'EyeTracking_sync_pulse_count',
    'BehaviorTracking_movie_frame_count',
    'BehaviorTracking_videomon_frame_count',
    'BehaviorTracking_sync_pulse_count',
]
error_types = [
    'likely_line_switch',
    'missing_eye_video_frames',
    'appears_fine_but_still_failing',
    'missing_vidoemon_log',
    'mystery_failure'
]
cols = cols + error_types
filtered_sessions_no_eye_tracking[cols].sort_values(by='date_of_acquisition').reset_index(drop=True)

,behavior_session_id,ophys_session_id,ophys_experiment_id,project_code,workflow_state,equipment_name,date_of_acquisition,EyeTracking_movie_frame_count,EyeTracking_videomon_frame_count,EyeTracking_sync_pulse_count,BehaviorTracking_movie_frame_count,BehaviorTracking_videomon_frame_count,BehaviorTracking_sync_pulse_count,likely_line_switch,missing_eye_video_frames,appears_fine_but_still_failing,missing_vidoemon_log,mystery_failure
0,778166125,778015591,[778644591],VisualBehavior,passed,CAM2P.5,2018-11-13 17:52:17.000000,144999,145003,145003,144976,144976,144976,False,True,False,False,False
1,783629024,783475910,[783927872],VisualBehavior,passed,CAM2P.4,2018-11-20 23:06:02.000000,144868,144873,144997,144997,144997,144873,False,False,False,False,True
2,784320573,784219195,[784482326],VisualBehavior,passed,CAM2P.4,2018-11-21 20:53:09.000000,145147,145147,145133,145133,145133,145147,True,False,False,False,False
3,788447502,788418567,[788489531],VisualBehavior,passed,CAM2P.5,2018-11-30 00:15:11.000000,145025,145035,145035,145027,145027,145027,False,True,False,False,False
4,792477679,792327341,[792813858],VisualBehavior,passed,CAM2P.4,2018-12-10 16:34:08.000000,135975,135975,135935,135935,135935,135975,True,False,False,False,False
5,792680306,792619807,[792815735],VisualBehavior,passed,CAM2P.4,2018-12-10 21:19:50.000000,136036,136036,136028,136028,136028,136036,True,False,False,False,False
6,794071128,793857113,[794381992],VisualBehavior,passed,CAM2P.4,2018-12-12 16:00:43.000000,135926,135926,135902,135902,135902,135926,True,False,False,False,False
7,794673280,794474159,[795076128],VisualBehavior,passed,CAM2P.4,2018-12-13 15:42:47.000000,136005,136005,135996,135996,135996,136005,True,False,False,False,False
8,794968300,794918442,[795073741],VisualBehavior,passed,CAM2P.4,2018-12-13 21:11:17.000000,135848,135848,135841,135841,135841,135848,True,False,False,False,False
9,795431009,795217244,[795952471],VisualBehavior,passed,CAM2P.4,2018-12-14 16:14:12.000000,136188,136188,136175,136175,136175,136188,True,False,False,False,False


In [163]:
filtered_sessions_no_eye_tracking.to_csv('/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/2020.05.07_sdk_issue_1545_log.csv', index=False)

In [153]:
filtered_sessions_no_eye_tracking[error_types].sum(axis=0)

likely_line_switch                21
missing_eye_video_frames          17
appears_fine_but_still_failing     6
missing_vidoemon_log               4
mystery_failure                    5
dtype: int64

In [152]:
filtered_sessions_no_eye_tracking[error_types].sum(axis=1).reset_index(drop=True)

0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    1
20    1
21    1
22    1
23    1
24    1
25    1
26    1
27    1
28    1
29    1
30    1
31    2
32    1
33    1
34    1
35    1
36    1
37    1
38    1
39    1
40    1
41    1
42    1
43    1
44    1
45    1
46    1
47    1
48    1
49    1
50    1
51    1
dtype: int64